In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import math
import os
import datetime
from pandas import DataFrame
import itertools
from bs4 import BeautifulSoup
import urllib.request
import requests
import random
import torch
import xmltodict
import xml.etree.ElementTree as ET
import xml
from xml.etree.ElementTree import parse
from xml.parsers import expat
from xml.dom.minidom import parse, parseString
import psycopg2
import PyPDF2
import pikepdf
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
import spacy

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
def createfiledirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return directory

# **LOAD & TRANSFORM BILL XML DATA FROM LOCAL FILE**

In [3]:
def iterroot(root, rootstr, elements):
    detlist=[]
    if rootstr in elements:
        for var in root.iter(rootstr):
            detlist.append(var.text)
    if detlist==[]:
        detlist=['']
    return detlist

In [4]:
def breaknames(name, xmlnames):
    first=''
    last=''
    state=''
    if name!='':
        if 'of ' in name:
            state=name.split('of ')[1];
            fullname=name.split(' of ')[0].split(' ')[1:];
        else:
            fullname=name.split(' ')[1:];
        if len(fullname)==2:
            first=fullname[0];
            last=fullname[1];
        elif len(fullname)==3:
            first=fullname[0]+ ' ' + fullname[1][0];
            last=fullname[2];
        elif len(fullname)==4:
            first=fullname[0]+ ' ' + fullname[1][0];
            last=fullname[2] + ' '+ fullname[3];
        elif len(fullname)==1:
            last=fullname[0];
    return first, last, state

In [5]:
def parsexml(xmlnamesraw, directoryxml):

    xmlnamesraw=xmlnamesraw[['Name', 'Link', 'OfficialTitle','LegisType', 'BillStage']]
    xmlnames=xmlnamesraw.set_index('Name')
    xmlspon=pd.DataFrame(columns=[ 'BillID', 'SponsorLastName', 'SponsorFirstName', 'SponsorState', 'PrimaryOrCoSponsor'])
    xmlcomm=pd.DataFrame(columns=[ 'BillID', 'CommittteeName'])
    xmldate=pd.DataFrame(columns=[ 'BillID', 'Date'])
    
    headerpd=pd.DataFrame(columns=['BillID', 'BillName', 'Header','HeaderID', 'HeaderOrderID', 'Key', 'Text',  ])
    billid=1
    headerid=0
    for billname in xmlnames.index:
        filename=directoryxml + '/' + billname + '.xml';
        tree = ET.parse(filename);
        root = tree.getroot();
        elements=[elem.tag for elem in root.iter()];
        xmlnames.loc[billname, 'OfficialTitle']=iterroot(root, 'official-title', elements)[0];
        xmlnames.loc[billname, 'LegisType']=iterroot(root,'legis-type', elements)[0];
        
        for var in root.iter('bill'):
            xmlnames.loc[billname, 'BillStage']=var.attrib['bill-stage'];
        
        date=iterroot(root,'action-date', elements)
        commit=iterroot(root,'committee-name',elements)
        spon=iterroot(root,'sponsor',elements)
        cospon=iterroot(root,'cosponsor',elements)
        
        for idval  in spon:
            first, last, state = breaknames(idval, xmlnames);
            xmlspon=xmlspon.append({ 'BillID':billid, 'SponsorLastName':last, 'SponsorFirstName':first, 'SponsorState':state, 
                                            'PrimaryOrCoSponsor':'Primary'}, ignore_index=True);
        for idval  in cospon:
            first, last, state = breaknames(idval, xmlnames);
            xmlspon=xmlspon.append({ 'BillID':billid, 'SponsorLastName':last, 'SponsorFirstName':first, 'SponsorState':state, 
                                            'PrimaryOrCoSponsor':'Co'}, ignore_index=True);
        for idval in date:
             xmldate=xmldate.append({'BillID':billid, 'Date':idval}, ignore_index=True);
        for idval in commit:
            xmlcomm=xmlcomm.append({'BillID':billid, 'CommittteeName':idval}, ignore_index=True);
      
        lasttag=''
        headerorder=0
        for var in root.iter('*'):
            tag=var.tag;
            text=var.text;
            if tag =='header':
                headerorder=1
                headerid=headerid+1
                lasttag='header';
                header=text;
                headerpd=headerpd.append({'BillID':billid, 'BillName': billname, 'Header':header, 'HeaderID':headerid, 'HeaderOrderID':headerorder,
                                          'Key':tag, 'Text':text}, ignore_index=True);
            elif tag=='paragraph':
                lasttag='paragraph';
            elif lasttag=='header' and tag in ['text', 'quote', 'short-title', 'external-xref', 'quoted-block', 'term'] and str(text)!='None':
                headerpd=headerpd.append({'BillID':billid, 'BillName': billname, 'Header':header, 
                                           'HeaderID':headerid, 'HeaderOrderID':headerorder, 'Key':tag, 'Text':text}, ignore_index=True);
                headerorder=headerorder+1
        billid=billid+1    
    return xmlnames,  xmldate,  xmlspon, xmlcomm, headerpd

In [6]:
def findkeywordlabel(headers):
    uniquewordpd=pd.DataFrame(columns=['WordText','WordTextLower', 'CategoryLabel'])
    nlp = spacy.load("en_core_web_sm")
    wordlabelpd=pd.DataFrame(columns=['BillID', 'HeaderID', 'HeaderOrderID',  'EntityOrderID', 'Text', 'CategoryLabel'])
    for ind in headers.index:
        doc = nlp(headers.loc[ind,'Text'])
        entorder=0
        for ent in doc.ents:
            entorder=entorder+1
            print(ent.text, ent.start_char, ent.end_char, ent.label_)
            s
            wordlabelpd=wordlabelpd.append({'BillID':headers.loc[ind, 'BillID'], 'HeaderID':headers.loc[ind, 'HeaderID'],'HeaderOrderID':headers.loc[ind, 'HeaderOrderID'],
                                            'EntityOrderID':entorder , 'HeaderText': ent.text, 'CategoryLabel': ent.label}, ignore_index=True)
            if ent.text.lower() not in uniquewordpd['WordTextLower'].values:
                uniquewordpd=uniquewordpd.append({'WordText': ent.text, 'WordTextLower': ent.text.lower(), 'CategoryLabel':ent.label}, ignore_index=True)
    return wordlabelpd, uniquewordpd

In [7]:
def findkeyword(headers):
    tokenpd=pd.DataFrame(columns=['BillID', 'HeaderID', 'HeaderOrderID','EntityOrderID', 'Text','De-Conjugate', 'POS Descr', 'POS', 'POS Text', 'Caps'])
    keeplist=['no', 'some', 'all','every','each', 'any', 'after', 'before']
    dellist= ['DET', 'ADP', 'CCONJ', 'PUNCT', 'SPACE'] 
    nlp = spacy.load("en_core_web_sm")
    for ind in headers.index:
        doc = nlp(headers.loc[ind,'Text'].replace('-', ' ').replace('     ', '')) 
        entorder=0
        for token in doc:
            if (token.pos_ not in dellist or  str(token.text).lower() in keeplist) :
                entorder=entorder+1
                tokenpd=tokenpd.append({ 'BillID': headers.loc[ind, 'BillID'], 'HeaderID': headers.loc[ind, 'headerID'],
                            'HeaderOrderID': headers.loc[ind, 'HeaderOrderID'],'EntityOrderID':entorder, 'Text':token.text, 'De-Conjugate': token.lemma_,
                            'POS Descr':token.pos_,  'POS':token.tag_, 'POS Text':token.dep_, 'Caps':token.shape_},ignore_index=True)
    return tokenpd

# **EXECUTE DATA IMPORT & TRANSFORM FUNCTIONS**

In [8]:
username='joythompson'
projfol="/Users/"+username+"/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/"
rawbilldf=projfol+ "Raw_Data_Files/Bill_Data_from_HTML_XML/"
directorybillmain=createfiledirectory(projfol+ 'Bill_Rep_Parsing_to_SQL_Main/')
directorybillhead=createfiledirectory(projfol+ 'Bill_Parsing_Headers/')
directoryxml=rawbilldf+ 'DSE_project_xml'
directorytext=rawbilldf+ 'DSE_project_text'
xmlnames = pd.read_csv(rawbilldf+ 'bill_attributes_via_xml_dataframe_only_index.csv')
textnames =pd.read_csv(rawbilldf+'bill_attributes_via_text_dataframe_only_index.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/joythompson/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/transitional_transform_data//bill_attributes_via_xml_dataframe_only_index.csv'

In [ ]:
xmlnames, xmldate,  xmlspon, xmlcomm, headerpd=parsexml(xmlnames, directoryxml)

In [ ]:
xmlnames.to_csv(directorybillmain+ 'bill_main_attributes.csv')
xmlspon.to_csv(directorybillmain+ 'bill_sponcospon_attributes.csv')
xmldate.to_csv(directorybillmain+ 'bill_date_attributes.csv')
xmlcomm.to_csv(directorybillmain+ 'bill_committee_attributes.csv')
headerpd.to_csv(directorybillhead+ 'header_text_by_bill_header_with_order.csv')